In [1]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd

import torch
from torch.autograd import Variable

from models import load_models_autoencoder, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [2]:
load_path='output/output_autoencoder1109'

In [4]:
model_args, idx2word, autoencoder,word2idx= load_models_autoencoder(load_path)

Loading models fromoutput/output_autoencoder1109


In [5]:
vocab2=idx2word
vocab2

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: '',
 5: '.',
 6: '1',
 7: '100%가',
 8: '100%보리로만',
 9: '100%보리맥주이고',
 10: '10캔',
 11: '15년',
 12: '15년쯤돼는대',
 13: '1로',
 14: '1위를',
 15: '1차로',
 16: '2.',
 17: '2002년',
 18: '20대',
 19: '20대는',
 20: '20대에',
 21: '20세',
 22: '23',
 23: '2ne1의',
 24: '2pm나오기도',
 25: '2pm이',
 26: '2pm팬이라서요',
 27: '2병마실것을',
 28: '2위를',
 29: '2차로',
 30: '30대인',
 31: '33%낮다고',
 32: '34병까지도',
 33: '3가지만',
 34: '3개월간',
 35: '3년간',
 36: '3분의',
 37: '50%이상',
 38: '500cc',
 39: '500년',
 40: 'Asahi',
 41: 'BI가',
 42: 'CASS라는',
 43: 'CF',
 44: 'CF가',
 45: 'CF가벼운맛.',
 46: 'CF광고가',
 47: 'CF를',
 48: 'CF모델이',
 49: 'CF에',
 50: 'CF에서',
 51: 'CF의',
 52: 'CL',
 53: 'CL양도',
 54: 'Cass',
 55: 'Cass라는',
 56: 'Cf의',
 57: 'GOOD.',
 58: 'Guiness',
 59: 'Hite',
 60: 'KGB',
 61: 'KGB가',
 62: 'KGB같은',
 63: 'KGB는',
 64: 'KGB레몬을',
 65: 'KGB를',
 66: 'KGB맥주를',
 67: 'KGB의',
 68: 'Light는',
 69: 'MAX라고',
 70: 'MAX만',
 71: 'Max',
 72: 'OB가',
 73: 'OB골든라거',
 74: 'OB라고',
 75: 'OB라는',
 7

In [6]:
word2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 '<oov>': 3,
 '': 4,
 '.': 5,
 '1': 6,
 '100%가': 7,
 '100%보리로만': 8,
 '100%보리맥주이고': 9,
 '10캔': 10,
 '15년': 11,
 '15년쯤돼는대': 12,
 '1로': 13,
 '1위를': 14,
 '1차로': 15,
 '2.': 16,
 '2002년': 17,
 '20대': 18,
 '20대는': 19,
 '20대에': 20,
 '20세': 21,
 '23': 22,
 '2ne1의': 23,
 '2pm나오기도': 24,
 '2pm이': 25,
 '2pm팬이라서요': 26,
 '2병마실것을': 27,
 '2위를': 28,
 '2차로': 29,
 '30대인': 30,
 '33%낮다고': 31,
 '34병까지도': 32,
 '3가지만': 33,
 '3개월간': 34,
 '3년간': 35,
 '3분의': 36,
 '50%이상': 37,
 '500cc': 38,
 '500년': 39,
 'Asahi': 40,
 'BI가': 41,
 'CASS라는': 42,
 'CF': 43,
 'CF가': 44,
 'CF가벼운맛.': 45,
 'CF광고가': 46,
 'CF를': 47,
 'CF모델이': 48,
 'CF에': 49,
 'CF에서': 50,
 'CF의': 51,
 'CL': 52,
 'CL양도': 53,
 'Cass': 54,
 'Cass라는': 55,
 'Cf의': 56,
 'GOOD.': 57,
 'Guiness': 58,
 'Hite': 59,
 'KGB': 60,
 'KGB가': 61,
 'KGB같은': 62,
 'KGB는': 63,
 'KGB레몬을': 64,
 'KGB를': 65,
 'KGB맥주를': 66,
 'KGB의': 67,
 'Light는': 68,
 'MAX라고': 69,
 'MAX만': 70,
 'Max': 71,
 'OB가': 72,
 'OB골든라거': 73,
 'OB라고': 74,
 'OB라는': 75,
 '

In [5]:
model_args

{'N': 5,
 'arch_d': '300-300',
 'arch_g': '300-300',
 'batch_size': 5,
 'beta1': 0.9,
 'clip': 1,
 'cuda': True,
 'data_path': 'data_korean',
 'dropout': 0.0,
 'emsize': 300,
 'enc_grad_norm': True,
 'epochs': 49,
 'gan_clamp': 0.01,
 'gan_toenc': -0.01,
 'hidden_init': False,
 'kenlm_path': '../Data/kenlm',
 'log_interval': 200,
 'lowercase': False,
 'lr_ae': 1,
 'lr_gan_d': 1e-05,
 'lr_gan_g': 5e-05,
 'maxlen': 30,
 'min_epochs': 50,
 'nhidden': 300,
 'niters_ae': 1,
 'niters_gan_d': 5,
 'niters_gan_g': 1,
 'niters_gan_schedule': '2-4-6',
 'nlayers': 1,
 'no_earlystopping': False,
 'noise_anneal': 0.995,
 'noise_radius': 0.2,
 'ntokens': 10919,
 'outf': 'out1102',
 'patience': 5,
 'sample': False,
 'seed': 1111,
 'temp': 1,
 'vocab_size': 11000,
 'z_size': 100}

# ---------전처리

In [266]:
import re

In [254]:
file = 'codedata/rawinput.csv'
rawdata = pd.read_csv(file, names = None, encoding='CP949')
rawdoc = rawdata['응답값']

In [265]:
rawdoc1 = [" ".join(re.findall('([가-힣]+)', row))+"." for row in rawdoc]
rawdoc1
with open("train.txt", 'w') as f:
    for s in rawdoc1:
        f.write(str(s) + '\n')

# ------------------

In [7]:
path = 'codedata/clusterdata.csv'
maxlen=30

In [8]:
data = pd.read_csv(path, names = None)
clusternum=data['ClusterNo']
doc = data['rawdoc']
groupby_cluster = data['rawdoc'].groupby([data['ClusterNo']])
cluster_doc=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

In [8]:
cluster_doc

{0: ['시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 어요 ~.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 아서

In [9]:
def doc_to_line(cluster_doc):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']
        # vectorize
        vocab = word2idx
        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]
#         print(indices)
        lines.append(indices)
        
    return lines, vocab

In [10]:
def make_code(lines, vocab):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    ntokens = len(vocab)
    
    
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder.encode(Variable(source), lengths, noise=False)
#             print(output)
            code.append(output)
    return code

In [11]:
cluster_title=[]
for c in cluster_doc : 
    lines, vocab = doc_to_line(cluster_doc[c])
    code = make_code(lines, vocab)

    code_cat=torch.cat(code,0)
    code_mean=torch.mean(code_cat,0).view(1,-1) #code 평균

    max_indices = autoencoder.generate_code(code_mean, maxlen)
    max_indices = max_indices.data.cpu().numpy().squeeze()
    

    words = [vocab2[x] for x in max_indices.squeeze()]
#     print(words)
    # truncate sentences to first occurrence of <eos>
    truncated_sent = []
    for w in words:
        if w != '<eos>':
            truncated_sent.append(w)
        else:
            break
    sent = " ".join(truncated_sent)
    cluster_title.append(sent)

In [12]:
len(cluster_title)

536

In [13]:
resultdoc={}
for i in range(len(cluster_title)):
    resultdoc[i]=cluster_title[i]

In [14]:
resultdoc

{0: '상큼해서.',
 1: '맛이 강하다.',
 2: '순한맛.',
 3: '맛있습니다.',
 4: '목넘김이 좋고 순해서.',
 5: '톡 쏘는 맛이 강하다.',
 6: '깔끔한맛.',
 7: '상큼한',
 8: '입맛에 입맛에 맞음.',
 9: '맛이 상큼해서.',
 10: '맛',
 11: '목넘김이 좋고',
 12: '맛이 깊고 크리미하다.',
 13: '그냥 친근하다.',
 14: '뒷맛이 개운하다.',
 15: '깔끔하고 맛있고 개운하다.',
 16: '맛이 순하고 개운하다.',
 17: '맥주 맛이 다르다.',
 18: '목넘김이 좋고 상쾌함.',
 19: '깔끔하고 뒷맛이 개운하다.',
 20: '부드럽고 고소하다.',
 21: '독특한 맛과 풍미.',
 22: '청량감 느껴지는 청량감.',
 23: '맛과 향이 독특함.',
 24: '순한 맛이 일품.',
 25: '상큼한 맛이 더있음.',
 26: '깨끗하고 부드러운 크림.',
 27: '시원한 맛이',
 28: '가장 맛이 있어요맛이.',
 29: '톡 쏘는 맛이 덜하고 상쾌함.',
 30: '깊은 맛 거품의',
 31: '보리 맛이 좋아요 상쾌하다.',
 32: '특유의 향과 맛과 일품.',
 33: '고급스럽고 고급스러운',
 34: '가격이 저렴하고 가격대비 맥주맛도 괜찮음.',
 35: '내 입맛에 가장 없고 국산',
 36: '호가든 맥주도 좋아하는 파란색이라서.',
 37: '자주 마시는 거라서.',
 38: '시원한 느낌이 좋타.',
 39: '상큼하고 맛있고 다양한맛이 산뜻함.',
 40: '부드러움 목넘김이 좋고 그리고 상쾌하다.',
 41: '시원한 맛 떄문.',
 42: '대중적이고 쉽게 접할 수 있고 레몬맛이여서.',
 43: '칼로리가 낮아서 칼로리가 없는',
 44: '제일 입맛에 맞고 가볍다.',
 45: '부드러우면 맛이 강하고 몸에 잘 나는 맥주다.',
 46: '뭔가 톡 쏘고 좋타.',
 47: '고소한 향과 거품의 일품.',
 48: '호가든 특유의 톡쏘

In [15]:
df = pd.DataFrame([resultdoc,cluster_doc]).T
df['clusterNo'] = df.index

In [16]:
df

,0,1,clusterNo
0,상큼해서.,"[시원해서., 시원함., 시원하다., 시원함., 시원해서., 시원함., 시원하다.,...",0
1,맛이 강하다.,"[맛이 깔끔하다., 맛이 깔끔함., 맛이 깔끔함., 깔끔한 맛., 맛이 깔끔하다.,...",1
2,순한맛.,"[부드럽다., 부드러움., 부드러움., 부드러움., 부드러움., 부드럽다., 부드러...",2
3,맛있습니다.,"[맛있다., 맛있음., 맛있어서., 맛있어서요., 맛있어요., 맛있다., 맛있어서....",3
4,목넘김이 좋고 순해서.,"[부드러운 목넘김이 좋다 맛도 부드러워 좋다., 목넘김이 좋고 부드럽다., 부드럽고...",4
5,톡 쏘는 맛이 강하다.,"[톡 쏘는 맛., 톡 쏘아서., 맛이 좋음 톡쏨., 톡 쏘는 맛이 좋다., 톡 쏘는...",5
6,깔끔한맛.,"[깔끔하다., 깔끔해요., 깔끔함., 깔끔함., 깔끔해서., 깔끔해서., 깔끔하므로...",6
7,상큼한,"[부드러운 맛., 맛이 부드러워서., 부드러운 맛., 맛이 부드럽다., 맛이 부드러...",7
8,입맛에 입맛에 맞음.,"[맛이 입맛에 맞아서., 내 입맛에 맞아서., 내 입맛에 맞음., 맛이 입맛에 맞아...",8
9,맛이 상큼해서.,"[맛이 좋다., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛...",9


In [17]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],row[0],  nn]) 
                         for nn in row[1]], axis=1)

In [18]:
df_new = pd.DataFrame(rows, columns=['clusterNo','title','raw_docs'])
df_new

,clusterNo,title,raw_docs
0,0,상큼해서.,시원해서.
1,0,상큼해서.,시원함.
2,0,상큼해서.,시원하다.
3,0,상큼해서.,시원함.
4,0,상큼해서.,시원해서.
5,0,상큼해서.,시원함.
6,0,상큼해서.,시원하다.
7,0,상큼해서.,시원함.
8,0,상큼해서.,시원해서.
9,0,상큼해서.,시원함.


In [19]:
df_new.to_csv('1109_generate_title_result_autoencoder.csv',sep=',',encoding='CP949')

# ------------------------------------------------

In [8]:
dropped = 0
with open(path, 'r') as f:
    linecount = 0
    lines = []
    for line in f:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']
        # vectorize
        vocab = word2idx
        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]
        lines.append(indices)

In [26]:
dropped = 0
linecount = 0
lines = []
for line in cluster_doc[48]:
    linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
    words = line[:-1].strip().split(" ")
    if len(words) > maxlen:
        dropped += 1
        continue
    words = ['<sos>'] + words
    words += ['<eos>']
    # vectorize
    vocab = word2idx
    unk_idx = vocab['<oov>']
    indices = [vocab[w] if w in vocab else unk_idx for w in words]
    lines.append(indices)

In [27]:
eval_batch_size = 1
test_data = batchify(lines, eval_batch_size, shuffle=False)
ntokens = len(word2idx)

In [28]:
code = []
for i, batch in enumerate(test_data):
        source, target, lengths = batch
#         source = to_gpu(True, Variable(source, volatile=True))
#         target = to_gpu(True, Variable(target, volatile=True))
        print(source)
        print(source.shape)
        print(lengths)
#         mask = target.gt(0)
#         masked_target = target.masked_select(mask)
#         # examples x ntokens
#         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

        # output: batch x seq_len x ntokens
        output = autoencoder.encode(Variable(source), lengths, noise=False)
        print(output)
        code.append(output)


    1  5020  3048
[torch.LongTensor of size 1x3]

torch.Size([1, 3])
[3]
Variable containing:

Columns 0 to 9 
-0.4109  0.2240 -0.5171  0.1784  0.3945 -0.5976  0.5891  0.3542  0.6336 -0.0092

Columns 10 to 19 
 0.8363 -0.1395 -0.1931 -0.2271 -0.6416 -0.0596  0.6023 -0.7724 -0.4374 -0.0137

Columns 20 to 29 
 0.3264  0.7486 -0.5704 -0.5601  0.0351 -0.5767  0.1553 -0.2205  0.1470  0.2975

Columns 30 to 39 
-0.0740  0.0715 -0.5254 -0.6654 -0.1387 -0.2738  0.1072 -0.8304  0.2127 -0.2477

Columns 40 to 49 
-0.0793  0.1631  0.5389  0.1282 -0.7743 -0.1925 -0.5989  0.1864  0.0158  0.4245

Columns 50 to 59 
 0.6548 -0.1518  0.5294  0.4486  0.7078 -0.1385 -0.1003  0.2794  0.0005  0.2516

Columns 60 to 69 
-0.1091 -0.2002 -0.6102  0.3125 -0.4647 -0.1504  0.4009  0.0655  0.1900 -0.0277

Columns 70 to 79 
 0.2422  0.4097  0.8396 -0.4333 -0.5609 -0.0061 -0.3958 -0.3646  0.2211  0.3314

Columns 80 to 89 
 0.2671 -0.3188  0.2231 -0.6381  0.2386  0.5157 -0.1798  0.5904  0.8183 -0.5584

Columns 90 to 9

Variable containing:

Columns 0 to 9 
-0.1550 -0.4113  0.6551  0.0284 -0.5073  0.2630  0.3073  0.0911  0.8431 -0.0792

Columns 10 to 19 
-0.1936  0.1043 -0.3395 -0.0547 -0.0875  0.0177  0.3064 -0.7259 -0.6650 -0.2415

Columns 20 to 29 
 0.3954  0.0096 -0.5428 -0.5185  0.1048  0.3067  0.0760 -0.0282  0.1056 -0.0461

Columns 30 to 39 
-0.0598  0.1959  0.4507  0.7181 -0.4160 -0.0853 -0.6801 -0.5405  0.2699 -0.2522

Columns 40 to 49 
-0.0587  0.4417  0.2349  0.0784 -0.1620  0.1943  0.1631 -0.2348  0.0405  0.1944

Columns 50 to 59 
 0.1971  0.0298 -0.3641  0.4219 -0.3220  0.2371  0.2962  0.0612 -0.1559  0.0632

Columns 60 to 69 
 0.0981 -0.3447  0.5208 -0.2452  0.5751 -0.3222  0.0228 -0.1334 -0.4839 -0.3997

Columns 70 to 79 
 0.1206  0.4908  0.1048 -0.3719 -0.2164 -0.0022 -0.6086 -0.2540  0.2380  0.0701

Columns 80 to 89 
 0.6078 -0.2986  0.7374 -0.2051  0.5014 -0.6803  0.5083 -0.1472 -0.7649 -0.2945

Columns 90 to 99 
-0.0105  0.0720 -0.3119 -0.1530 -0.6335  0.0172 -0.5179  0.3648  0.7628

In [29]:
code_mean=torch.max(code_cat,0)[0].view(1,-1)

In [30]:
code_cat=torch.cat(code,0)
code_mean=torch.mean(code_cat,0).view(1,-1)

In [31]:
max_indices = autoencoder.generate_code(code_mean, maxlen)
max_indices = max_indices.data.cpu().numpy().squeeze()

In [32]:
words = [vocab2[x] for x in max_indices.squeeze()]
print(words)
# truncate sentences to first occurrence of <eos>
truncated_sent = []
for w in words:
    if w != '<eos>':
        truncated_sent.append(w)
    else:
        break
sent = " ".join(truncated_sent)

['호가든', '특유의', '과일향과', '<eos>', '<eos>', '<eos>', '<eos>', '맛이', '느껴져서.', '<eos>', '<eos>', '<eos>', '향과', '거품도', '느껴져서.', '<eos>', '<eos>', '<eos>', '향과', '맛이', '느껴져서.', '<eos>', '<eos>', '<eos>', '특유의', '청량감.', '<eos>', '<eos>', '<eos>', '톡쏘는']


In [221]:
sent

'카스가 대표적이니까.'